In [1]:
import pandas as pd
import numpy as np

import regex as re

from sklearn.impute import KNNImputer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('data.csv')
df.head()

,Property,Bhk,Floor,Area,Price
0,Flat,3,NaN,957.0,₹\n1.92 Cr
1,Flat,3,NaN,1199.0,₹\n1.29 Cr
2,Flat,2,NaN,882.0,₹\n1.65 Cr
3,Flat,2,NaN,825.0,₹\n91 Lac
4,Flat,3,NaN,1414.0,₹\n2.97 Cr


In [3]:
df.shape

(3150, 5)

In [4]:
print("Property = ",df['Property'].unique())
print("BHK = ",df['Bhk'].unique())
print("Floor = ",df['Floor'].unique())

Property =  ['Flat' 'Apartment' 'Builder' 'Penthouse']
BHK =  [3 2 4 1 5]
Floor =  [nan '14' '11' '30' '5' '8' '10' '12' '9' '15' '18' '16' '7' '4' '6' '3'
 'Lower' '22' '2' '1' '20' '21' '17' '23' 'Ground' '19' '27' '26' '13'
 'Upper' '29' '28']


In [5]:
df['Floor'] = df['Floor'].replace('Lower',0)
df['Floor'] = df['Floor'].replace('Upper',1)
df['Floor'] = df['Floor'].replace('Ground',0)

In [6]:
df.isna().sum()

Property      0
Bhk           0
Floor       792
Area        133
Price         0
dtype: int64

In [7]:
df['Area'].fillna(df['Area'].median(),inplace = True)

In [8]:
# Flat = 1, Apartment = 2, Builder = 3, Penthouse = 4
df['Property'] = df['Property'].replace('Flat',1)
df['Property'] = df['Property'].replace('Apartment',2)
df['Property'] = df['Property'].replace('Builder',3)
df['Property'] = df['Property'].replace('Penthouse',4)

In [9]:
def convert_to_float(s):
    digits = re.findall(r'\d+', s)
    val = ''
    for num in digits:
        val += (num) + '.'
    if s[-2:].strip() == "Cr":
        return float(val[:-1]) * 10000000
    else:
        return float(val[:-1]) * 100000

df['Price'] = df['Price'].apply(lambda x: convert_to_float(x))

In [10]:
imputer = KNNImputer(n_neighbors = 3)
df = pd.DataFrame(imputer.fit_transform(df),columns = df.columns)

In [11]:
df.head()

,Property,Bhk,Floor,Area,Price
0,1.0,3.0,4.0,957.0,19200000.0
1,1.0,3.0,11.0,1199.0,12900000.0
2,1.0,2.0,5.0,882.0,16500000.0
3,1.0,2.0,5.0,825.0,9100000.0
4,1.0,3.0,9.0,1414.0,29700000.0


In [12]:
X = df.drop('Price',axis = 1)
y = df['Price']

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)
lr = LinearRegression()
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

0.757982278022182

In [14]:
rfr = RandomForestRegressor()
rfr.fit(X_train,y_train)
rfr.score(X_test,y_test)

0.9901423109009297

In [15]:
dtr = DecisionTreeRegressor()
dtr.fit(X_train,y_train)
dtr.score(X_test,y_test)

0.9917121146376804